In [1]:
import pandas as pd
import json

def load_data(path):
    with open(path, 'r') as file:
        data = file.read()

    data = data.strip().split('\n')

    dataset = []
    for i in data:
        try:
            dataset.append(json.loads(i))
        except:
            print(i)

    df = pd.DataFrame(dataset)

    return df

TRAIN = load_data('data/train/train.json')
TRAIN


,topic,question,excerpt
0,electronics,What is the effective differencial effective o...,"I'm trying to work out, in general terms, the ..."
1,electronics,Heat sensor with fan cooling,Can I know which component senses heat or acts...
2,electronics,Outlet Installation--more wires than my new ou...,I am replacing a wall outlet with a Cooper Wir...
3,electronics,Buck Converter Operation Question,"i have been reading about the buck converter, ..."
4,electronics,"Urgent help in area of ASIC design, verificati...",I need help with deciding on a Master's Projec...
...,...,...,...
20214,wordpress,How to set a Custom Post Type as the parent of...,I have a Custom Post Type called Recipe with p...
20215,wordpress,Tracking last login and last visit,I'm using the code below to track when a user ...
20216,wordpress,How to exclude the particular category from th...,"add_action( 'pre_get_posts', 'custom_pre_get_p..."
20217,wordpress,display sub categories assoccited with each po...,i have wordpress blog with many posts. each po...


In [2]:
TRAIN.columns

Index(['topic', 'question', 'excerpt'], dtype='object')

In [3]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK resources
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')

In [4]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    """Function to clean text and perform lemitisation"""
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    words = word_tokenize(text)
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return " ".join(words)

In [5]:
df = TRAIN.copy()
df['text'] = df['question'] + " " + df['excerpt']
df['text'] = df['text'].apply(preprocess_text)
df = df[['text', 'topic']]

In [6]:
df['text'][0]

'effective differencial effective circuit im trying work general term effective capacitance circuit see diagram httpistackimgurcombs85bpng effective capacitance circuit'

## TF-IDF Vectorizer Methord

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

def vectorirse_text(text):
    """ Recieves text as input and returns TF-IDF vectors"""
    tfidf = TfidfVectorizer(max_features=500000)
    X = tfidf.fit_transform(text)
    return X

def label_encoding(input):
    label_encoder = LabelEncoder()
    return label_encoder.fit_transform(input)


X = vectorirse_text(df['text'])
y = label_encoding(df['topic'])



In [ ]:
from models import *
print("Logistic Regression:")
report, acc = logistic_regression(X, y)
print("Accuracy:", acc)
print(report)

print("\nDecision Tree Classifier:")
report, acc = decision_tree(X, y)
print("Accuracy:", acc)
print(report)

print("\nRandom Forest Classifier:")
report, acc = random_forest(X, y)
print("Accuracy:", acc)
print(report)

print("\nSupport Vector Machine (SVM):")
report, acc = support_vector_machine(X, y)
print("Accuracy:", acc)
print(report)

print("\nk-Nearest Neighbors (k-NN):")
report, acc = knn(X, y, k=5)
print("Accuracy:", acc)
print(report)

print("\nNaïve Bayes Classifier:")
report, acc = naive_bayes(X, y)
print("Accuracy:", acc)
print(report)

print("\nGradient Boosting (XGBoost):")
report, acc = xgboost_classifier(X, y)
print("Accuracy:", acc)
print(report)

print("\nMulti-Layer Perceptron (MLP - Neural Network):")
report, acc = mlp_classifier(X, y)

print("Accuracy:", acc)
print(report)



Multi-Layer Perceptron (MLP - Neural Network):
Accuracy: 0.7925321463897131
              precision    recall  f1-score   support

           0       0.80      0.77      0.79       458
           1       0.61      0.72      0.66       396
           2       0.91      0.80      0.85       363
           3       0.75      0.92      0.83       497
           4       0.73      0.74      0.73       294
           5       0.83      0.84      0.84       368
           6       0.99      0.82      0.89       498
           7       0.74      0.78      0.76       388
           8       0.73      0.66      0.69       396
           9       0.90      0.83      0.86       386

    accuracy                           0.79      4044
   macro avg       0.80      0.79      0.79      4044
weighted avg       0.80      0.79      0.79      4044



## Word2Vector Methord

In [31]:
import gensim
import numpy as np
from sklearn.ensemble import RandomForestClassifier

df['tokens'] = df['text'].apply(lambda x: x.split())

w2v_model = gensim.models.Word2Vec(sentences=df['tokens'], vector_size=10000, window=5, min_count=2, workers=10)

def get_sentence_embedding(tokens):
    vectors = [w2v_model.wv[word] for word in tokens if word in w2v_model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(100)

X_w2v = np.array(df['tokens'].apply(get_sentence_embedding).tolist())
y_encoded = label_encoding(df['topic'])


In [32]:
type(X_w2v)

numpy.ndarray

In [33]:
X = X_w2v
y = y_encoded

In [34]:
print("Logistic Regression:")
report, acc = logistic_regression(X, y)
print("Accuracy:", acc)
print(report)

print("\nDecision Tree Classifier:")
report, acc = decision_tree(X, y)
print("Accuracy:", acc)
print(report)

print("\nRandom Forest Classifier:")
report, acc = random_forest(X, y)
print("Accuracy:", acc)
print(report)

print("\nSupport Vector Machine (SVM):")
report, acc = support_vector_machine(X, y)
print("Accuracy:", acc)
print(report)

print("\nk-Nearest Neighbors (k-NN):")
report, acc = knn(X, y, k=5)
print("Accuracy:", acc)
print(report)

print("\nNaïve Bayes Classifier:")
report, acc = naive_bayes(X, y)
print("Accuracy:", acc)
print(report)

print("\nGradient Boosting (XGBoost):")
report, acc = xgboost_classifier(X, y)
print("Accuracy:", acc)
print(report)

print("\nMulti-Layer Perceptron (MLP - Neural Network):")
report, acc = mlp_classifier(X, y)

print("Accuracy:", acc)
print(report)

Logistic Regression:


/home/darth/.pyenv/versions/3.10.12/envs/major02/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy: 0.6434223541048467
              precision    recall  f1-score   support

           0       0.78      0.74      0.76       458
           1       0.52      0.48      0.50       396
           2       0.56      0.68      0.61       363
           3       0.68      0.69      0.69       497
           4       0.56      0.53      0.55       294
           5       0.73      0.75      0.74       368
           6       0.70      0.79      0.74       498
           7       0.57      0.49      0.53       388
           8       0.50      0.48      0.49       396
           9       0.76      0.70      0.73       386

    accuracy                           0.64      4044
   macro avg       0.64      0.63      0.63      4044
weighted avg       0.64      0.64      0.64      4044


Decision Tree Classifier:
Accuracy: 0.5054401582591493
              precision    recall  f1-score   support

           0       0.68      0.62      0.65       458
           1       0.37      0.39      0.38    

ValueError: Negative values in data passed to MultinomialNB (input X).

In [11]:
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from tqdm import tqdm

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load pre-trained BERT tokenizer & model on GPU
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)  # Move model to GPU

# Function to get BERT embeddings using GPU
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(device)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()  # Move result back to CPU for NumPy

# Convert text to BERT embeddings with progress bar
X_bert = np.array([get_bert_embedding(text) for text in tqdm(df['text'], desc="Generating BERT Embeddings on GPU")])


# Train-test split
X_train_bert, X_test_bert, y_train_bert, y_test_bert = train_test_split(X_bert, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

# Train Logistic Regression
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train_bert, y_train_bert)

# Predict & Evaluate
y_pred_bert = lr_model.predict(X_test_bert)
bert_acc = accuracy_score(y_test_bert, y_pred_bert)
print(f"BERT + Logistic Regression Accuracy: {bert_acc:.4f}")


/home/darth/.pyenv/versions/3.10.12/envs/major02/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


Generating BERT Embeddings on GPU: 100%|██████████| 20219/20219 [02:22<00:00, 141.59it/s]


BERT + Logistic Regression Accuracy: 0.8029


In [12]:
TRAIN.columns

Index(['topic', 'question', 'excerpt'], dtype='object')

In [ ]:
df['x_bert']=X_bert.tolist()
df['x_w2v']=X_w2v.tolist()
df['x_tfidf']=X.toarray().tolist()
df['question']=TRAIN['question']
df['excerpt']=TRAIN['excerpt']
df.to_csv('df.csv', index=False)

In [ ]:
# import torch
# from transformers import BertTokenizer, BertModel
# import numpy as np
# from tqdm import tqdm

# # Check if GPU is available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# # Load pre-trained BERT tokenizer & model on GPU
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)  # Move model to GPU

# # Function to get BERT embeddings using GPU
# def get_bert_embedding(text):
#     inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(device)
#     with torch.no_grad():
#         outputs = bert_model(**inputs)
#     return outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()  # Move result back to CPU for NumPy

# # Convert text to BERT embeddings with progress bar
# X_bert = np.array([get_bert_embedding(text) for text in tqdm(df['text'], desc="Generating BERT Embeddings on GPU")])


Using device: cuda


KeyboardInterrupt: 

## GloVe Embeddings Method

In this section, we will use GloVe (Global Vectors for Word Representation) embeddings to convert text data into numerical vectors. GloVe is an unsupervised learning algorithm for obtaining vector representations for words. Training is performed on aggregated global word-word co-occurrence statistics from a corpus, and the resulting representations showcase interesting linear substructures of the word vector space.

We will follow these steps:
1. Load pre-trained GloVe embeddings.
2. Create a function to convert text into GloVe embeddings.
3. Apply the function to our dataset to obtain GloVe embeddings for each text.
4. Train and evaluate machine learning models using these embeddings.

In [7]:
import numpy as np
import gensim.downloader as api

# Load Pretrained GloVe Model
glove_model = api.load("glove-wiki-gigaword-300")  

# Convert a sentence to a vector by averaging word embeddings
def get_sentence_embedding(sentence, model, dim=300):
    words = sentence.split()
    word_vectors = [model[word] for word in words if word in model]
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(dim)

text_samples = ["GloVe embeddings capture word semantics"]
embeddings = np.array([get_sentence_embedding(text, glove_model) for text in text_samples])


In [8]:
from tqdm import tqdm

# Generate GloVe embeddings for the entire dataset with a progress bar
df['glove_embeddings'] = df['text'].apply(lambda x: get_sentence_embedding(x, glove_model))

# Convert the list of embeddings to a numpy array
X_glove = np.array([get_sentence_embedding(text, glove_model) for text in tqdm(df['text'], desc="Generating GloVe Embeddings")])

# Check the shape of the generated embeddings
print(X_glove.shape)

Generating GloVe Embeddings: 100%|██████████| 20219/20219 [00:00<00:00, 26234.90it/s]

(20219, 300)


In [9]:
df['glove_embeddings']

0        [-0.115741186, -0.20021904, 0.12441649, -0.324...
1        [-0.0010313676, -0.06625991, 0.31336138, -0.32...
2        [-0.08451371, -0.041777406, -0.17886283, -0.18...
3        [-0.1282833, 0.19806932, 0.16693015, -0.017683...
4        [-0.073781654, 0.05777748, -0.10044446, -0.301...
                               ...                        
20214    [-0.36480942, 0.14811745, 0.0026721125, -0.169...
20215    [-0.12177454, 0.06218006, -0.05326008, -0.0159...
20216    [-0.02713412, 0.04559438, 0.006360337, -0.1446...
20217    [-0.29473746, 0.2867133, -0.12950301, -0.03180...
20218    [-0.3100589, 0.15244281, 0.0920101, -0.0856748...
Name: glove_embeddings, Length: 20219, dtype: object

In [13]:
X = df['glove_embeddings']
y = label_encoding(df['topic']) 


In [14]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

X_sigmoid = sigmoid(X)


TypeError: loop of ufunc does not support argument 0 of type numpy.ndarray which has no callable exp method

In [15]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# X_sigmoid = sigmoid(X)

X_s = X.apply(lambda emb: sigmoid(np.array(emb)))


In [19]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Apply sigmoid function element-wise to each embedding
X_sigmoid = X.apply(lambda emb: np.array([sigmoid(val) for val in emb]))

# If you need it as a DataFrame:
X_sigmoid_df = pd.DataFrame(X_sigmoid.tolist())

print(X_sigmoid_df.head())  # Display first few rows

df['glove_sigmoid']=X_sigmoid


        0         1         2         3         4         5         6    \
0  0.471097  0.450112  0.531064  0.419600  0.529166  0.501688  0.539391   
1  0.499742  0.483441  0.577706  0.419251  0.488925  0.483124  0.525759   
2  0.478884  0.489557  0.455403  0.454474  0.504673  0.511277  0.525448   
3  0.467973  0.549356  0.541636  0.495579  0.533500  0.510909  0.493776   
4  0.481563  0.514440  0.474910  0.425199  0.494169  0.486540  0.507572   

        7         8         9    ...       290       291       292       293  \
0  0.504431  0.471951  0.210885  ...  0.484047  0.508446  0.468081  0.511043   
1  0.498380  0.512296  0.295908  ...  0.508435  0.516001  0.475152  0.510036   
2  0.496180  0.487645  0.263449  ...  0.493034  0.459211  0.474147  0.547312   
3  0.516288  0.510662  0.222204  ...  0.497405  0.476240  0.471470  0.508767   
4  0.513841  0.517520  0.228781  ...  0.491363  0.455957  0.501189  0.502150   

        294       295       296       297       298       299  
0  0

In [20]:
X = df['glove_sigmoid']
y = label_encoding(df['topic'])

In [21]:
from models import *

In [22]:
# print("Logistic Regression:")
# report, acc = logistic_regression(X, y)
# print("Accuracy:", acc)
# print(report)

# print("\nDecision Tree Classifier:")
# report, acc = decision_tree(X, y)
# print("Accuracy:", acc)
# print(report)

# print("\nRandom Forest Classifier:")
# report, acc = random_forest(X, y)
# print("Accuracy:", acc)
# print(report)

# print("\nSupport Vector Machine (SVM):")
# report, acc = support_vector_machine(X, y)
# print("Accuracy:", acc)
# print(report)

# print("\nk-Nearest Neighbors (k-NN):")
# report, acc = knn(X, y, k=5)
# print("Accuracy:", acc)
# print(report)

# print("\nNaïve Bayes Classifier:")
# report, acc = naive_bayes(X, y)
# print("Accuracy:", acc)
# print(report)

# print("\nGradient Boosting (XGBoost):")
# report, acc = xgboost_classifier(X, y)
# print("Accuracy:", acc)
# print(report)

print("\nMulti-Layer Perceptron (MLP - Neural Network):")
report, acc = mlp_classifier(X, y)

print("Accuracy:", acc)
print(report)


Multi-Layer Perceptron (MLP - Neural Network):


ValueError: setting an array element with a sequence.

In [23]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 MB 5.9 MB/s eta 0:00:0000:0100:02
  Using cached gast-0.6.0-py3-none-any.whl (21 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 11.1 MB/s eta 0:00:0000:0100:01
  Using cached protobuf-5.29.3-cp38-abi3-manylinux2014_x86_64.whl (319 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl (24.5 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached termcolor-2.5.0-py3-none-any.whl (7.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.7 MB/s eta 0:00:00a 0:00:01
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached tensorflow_io_gcs_filesystem-0.37.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 11.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 13.7 MB/s eta 0:00:00
  Usin

In [37]:
X.shape

(20219, 300)

In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Convert data to NumPy arrays (if not already)
X = np.array(X)  # Ensure X is a NumPy array
y = np.array(y)  # Convert labels to NumPy array

# Encode labels to integers (0-9)
unique_labels = np.unique(y)
label_map = {label: idx for idx, label in enumerate(unique_labels)}
y_encoded = np.array([label_map[label] for label in y])

# One-hot encode labels
y_one_hot = np.eye(10)[y_encoded]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

# Create DataLoader for efficient training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the Neural Network Model
class NeuralNet(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, num_classes)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.softmax(self.fc5(x))
        return x

# Initialize model, loss function, and optimizer
model = NeuralNet(input_size=X.shape[1], num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 100
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}")

# Evaluate the model
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        _, predicted = torch.max(outputs, 1)
        _, labels = torch.max(batch_y, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f"\nTest Accuracy: {accuracy:.4f}")

# Predict on new data
sample_X = X_test_tensor[:5]
model.eval()
predictions = model(sample_X)
predicted_classes = torch.argmax(predictions, dim=1).cpu().numpy()
predicted_labels = [unique_labels[idx] for idx in predicted_classes]
print("\nPredicted Labels:", predicted_labels)


(16175, 300) (4044, 300) (16175, 10) (4044, 10)
Epoch [1/100], Loss: 2.2988
Epoch [2/100], Loss: 2.2972
Epoch [3/100], Loss: 2.3108
Epoch [4/100], Loss: 2.3176
Epoch [5/100], Loss: 2.2910
Epoch [6/100], Loss: 2.2063
Epoch [7/100], Loss: 2.1915
Epoch [8/100], Loss: 2.1855
Epoch [9/100], Loss: 2.1844
Epoch [10/100], Loss: 2.1815
Epoch [11/100], Loss: 2.1742
Epoch [12/100], Loss: 2.1727
Epoch [13/100], Loss: 2.1802
Epoch [14/100], Loss: 2.1745
Epoch [15/100], Loss: 2.1711
Epoch [16/100], Loss: 2.1736
Epoch [17/100], Loss: 2.1669
Epoch [18/100], Loss: 2.1763
Epoch [19/100], Loss: 2.1600
Epoch [20/100], Loss: 2.1624
Epoch [21/100], Loss: 2.1802
Epoch [22/100], Loss: 2.1658
Epoch [23/100], Loss: 2.1810
Epoch [24/100], Loss: 2.1584
Epoch [25/100], Loss: 2.1622
Epoch [26/100], Loss: 2.1523
Epoch [27/100], Loss: 2.1576
Epoch [28/100], Loss: 2.1665
Epoch [29/100], Loss: 2.1527
Epoch [30/100], Loss: 2.1565
Epoch [31/100], Loss: 2.1611
Epoch [32/100], Loss: 2.1486
Epoch [33/100], Loss: 2.1582
Epoc

# Albert Embeddings

In [7]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import AlbertTokenizer, AlbertModel

# Load ALBERT tokenizer and model
tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")
model = AlbertModel.from_pretrained("albert-base-v2")

# Set device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load your dataset (Ensure df['text'] contains the text data)
# df = pd.read_csv("your_dataset.csv")  # Replace with your dataset file
texts = df["text"].tolist()

# Function to generate embeddings
def get_albert_embeddings(texts, batch_size=16):
    model.eval()  # Set model to evaluation mode
    embeddings_list = []

    for i in tqdm(range(0, len(texts), batch_size), desc="Processing"):
        batch_texts = texts[i : i + batch_size]
        
        # Tokenize the batch
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = {key: val.to(device) for key, val in inputs.items()}  # Move to GPU/CPU
        
        # Get embeddings
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Extract [CLS] token representation (sentence embedding)
        cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        embeddings_list.append(cls_embeddings)

    return np.vstack(embeddings_list)  # Stack all embeddings

# Generate embeddings for all texts
embeddings = get_albert_embeddings(texts)

/home/darth/.pyenv/versions/3.10.12/envs/major02/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-18 22:45:59.369565: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-18 22:45:59.377294: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742318159.387742   95942 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742318159.392140   95942 

In [10]:
X = embeddings
y = label_encoding(df['topic'])

In [14]:
from models import *

print("Logistic Regression:")
report, acc = logistic_regression(X, y)
print("Accuracy:", acc)
print(report)

print("\nDecision Tree Classifier:")
report, acc = decision_tree(X, y)
print("Accuracy:", acc)
print(report)

print("\nRandom Forest Classifier:")
report, acc = random_forest(X, y)
print("Accuracy:", acc)
print(report)

print("\nSupport Vector Machine (SVM):")
report, acc = support_vector_machine(X, y)
print("Accuracy:", acc)
print(report)

print("\nk-Nearest Neighbors (k-NN):")
report, acc = knn(X, y, k=5)
print("Accuracy:", acc)
print(report)

# print("\nNaïve Bayes Classifier:")
# report, acc = naive_bayes(X, y)
# print("Accuracy:", acc)
# print(report)

print("\nGradient Boosting (XGBoost):")
report, acc = xgboost_classifier(X, y)
print("Accuracy:", acc)
print(report)

# print("\nMulti-Layer Perceptron (MLP - Neural Network):")
# report, acc = mlp_classifier(X, y)

print("Accuracy:", acc)
print(report)

Logistic Regression:


/home/darth/.pyenv/versions/3.10.12/envs/major02/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/darth/.pyenv/versions/3.10.12/envs/major02/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.755192878338279
              precision    recall  f1-score   support

           0       0.75      0.74      0.75       458
           1       0.63      0.59      0.61       396
           2       0.76      0.77      0.76       363
           3       0.78      0.76      0.77       497
           4       0.71      0.69      0.70       294
           5       0.84      0.88      0.86       368
           6       0.94      0.95      0.95       498
           7       0.71      0.71      0.71       388
           8       0.55      0.57      0.56       396
           9       0.82      0.84      0.83       386

    accuracy                           0.76      4044
   macro avg       0.75      0.75      0.75      4044
weighted avg       0.75      0.76      0.75      4044


Decision Tree Classifier:
Accuracy: 0.3508902077151335
              precision    recall  f1-score   support

           0       0.31      0.31      0.31       458
           1       0.23      0.24      0.23     

/home/darth/.pyenv/versions/3.10.12/envs/major02/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [23:00:38] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.6923837784371909
              precision    recall  f1-score   support

           0       0.67      0.68      0.67       458
           1       0.58      0.49      0.53       396
           2       0.69      0.69      0.69       363
           3       0.66      0.71      0.68       497
           4       0.69      0.61      0.65       294
           5       0.78      0.84      0.81       368
           6       0.94      0.92      0.93       498
           7       0.62      0.61      0.62       388
           8       0.51      0.54      0.53       396
           9       0.74      0.75      0.74       386

    accuracy                           0.69      4044
   macro avg       0.69      0.69      0.69      4044
weighted avg       0.69      0.69      0.69      4044

Accuracy: 0.6923837784371909
              precision    recall  f1-score   support

           0       0.67      0.68      0.67       458
           1       0.58      0.49      0.53       396
           2       0